<a href="https://colab.research.google.com/github/AkashKoley012/Computer-Vision-Projects/blob/main/UTKFace.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
 99% 329M/331M [00:15<00:00, 21.7MB/s]
100% 331M/331M [00:15<00:00, 22.2MB/s]


In [2]:
import zipfile
with zipfile.ZipFile('utkface-new.zip', 'r') as zip_ref:
    zip_ref.extractall('utkface')

In [38]:
import os
import cv2
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

import torch
import torch.nn as nn
from PIL import Image
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader

In [4]:
os.remove('utkface-new.zip')

In [5]:
images = []
ages = []
genders = []
ethnics = []

for file in tqdm(os.listdir('/content/utkface/UTKFace')):
  # Split the filename by underscore and check if it has at least 3 parts
  parts = file.split('_')
  if len(parts) > 3:
    try:
      img = cv2.imread('/content/utkface/UTKFace/' + file)
      img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
      images.append(img)

      age, gender, ethnicity = parts[:3]
      ages.append(int(age))
      genders.append(int(gender))
      ethnics.append(int(ethnicity))
    except ValueError:
      print(f"Skipping file with unexpected format: {file}")
  else:
    print(f"Skipping file with unexpected format: {file}")

 13%|█▎        | 3146/23708 [00:01<00:06, 2977.44it/s]

Skipping file with unexpected format: 61_1_20170109150557335.jpg.chip.jpg


 92%|█████████▏| 21840/23708 [00:07<00:00, 3311.56it/s]

Skipping file with unexpected format: 39_1_20170116174525125.jpg.chip.jpg


100%|██████████| 23708/23708 [00:08<00:00, 2806.10it/s]

Skipping file with unexpected format: 61_1_20170109142408075.jpg.chip.jpg


In [6]:
print(len(images), len(ages), len(genders), len(ethnics))

23705 23705 23705 23705


In [7]:
df = pd.DataFrame({'image': images, 'age': ages, 'gender': genders, 'ethnicity': ethnics})

In [8]:
df.shape

(23705, 4)

In [9]:
split = int(len(df) * 0.8)
train_df = df[:split]
test_df = df[split:]

# Dataset & DataLoader

In [10]:
class UTKFaceDataset(Dataset):
  def __init__(self, df, transform=None):
    self.df = df
    self.transform = transform
  def __len__(self):
    return len(self.df)
  def __getitem__(self, idx):
    image = self.df.iloc[idx]['image']
    age = self.df.iloc[idx]['age']
    gender = self.df.iloc[idx]['gender']
    ethnicity = self.df.iloc[idx]['ethnicity']

    # Convert the NumPy array to a PIL Image before applying transforms
    image = Image.fromarray(image)

    if self.transform:
      image = self.transform(image)

    age = torch.tensor(age, dtype=torch.float32)
    gender = torch.tensor(gender, dtype=torch.float32)
    ethnicity = torch.tensor(ethnicity, dtype=torch.long)
    return image, age, gender, ethnicity

In [11]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
train_dataset = UTKFaceDataset(train_df, transform=transform)
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = UTKFaceDataset(test_df, transform=transform)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [13]:
for images, age_labels, gender_labels, ethnicity_labels in train_dataloader:
  print(images.shape, age_labels.shape, gender_labels.shape, ethnicity_labels.shape)
  break

torch.Size([64, 3, 224, 224]) torch.Size([64]) torch.Size([64]) torch.Size([64])


In [36]:
for images, age_labels, gender_labels, ethnicity_labels in test_dataloader:
  print(age_labels.float().unsqueeze(1).shape)
  break

torch.Size([64, 1])


# Model use ResNet-152

In [26]:
class AgeGenderEthnicityResNet(nn.Module):
  def __init__(self):
    super().__init__()
    self.resnet = models.resnet152(pretrained=True)
    self.resnet.fc = nn.Identity()
    self.fc1 = nn.Linear(2048, 1024)
    self.fc2 = nn.Linear(1024, 512)

    self.age_output = nn.Linear(512, 1)
    self.gender_output = nn.Linear(512, 1)
    self.ethnicity_output = nn.Linear(512, 5)

  def forward(self, x):
    x = self.resnet(x)
    x = self.fc1(x)

    age_x = self.fc2(x)
    age = self.age_output(age_x)

    gender_x = self.fc2(x)
    gender = torch.sigmoid(self.gender_output(gender_x))

    ethnicity_x = self.fc2(x)
    ethnicity = self.ethnicity_output(ethnicity_x)

    return age, gender, ethnicity


In [27]:
lr = 0.001

# Initialize Model
model = AgeGenderEthnicityResNet()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Loss functions
criterion_age = nn.MSELoss()
criterion_gender = nn.BCELoss()
criterion_ethnicity = nn.CrossEntropyLoss()

# Optimizer
optimizer = optim.Adam(model.parameters(), lr=lr)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [28]:
# Freeze resnet layers (feature extractor)
for param in model.resnet.parameters():
    param.requires_grad = False

In [29]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

2626567

In [30]:
model

AgeGenderEthnicityResNet(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequen

In [37]:
epochs = 1

# Training Loop
for epoch in tqdm(range(epochs)):
    model.train()
    running_loss = 0.0
    for images, age_labels, gender_labels, ethnicity_labels in train_dataloader:
        images, age_labels, gender_labels, ethnicity_labels = (
            images.to(device),
            age_labels.float().unsqueeze(1).to(device),
            gender_labels.float().unsqueeze(1).to(device),
            ethnicity_labels.to(device)
        )

        optimizer.zero_grad()

        age_preds, gender_preds, ethnicity_preds = model(images)
        loss_age = criterion_age(age_preds, age_labels)
        loss_gender = criterion_gender(gender_preds, gender_labels)
        loss_ethnicity = criterion_ethnicity(ethnicity_preds, ethnicity_labels)

        loss = loss_age + loss_gender + loss_ethnicity

        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_dataloader)}")



100%|██████████| 1/1 [00:00<00:00,  3.15it/s]

torch.Size([64, 1]) torch.Size([64, 1])
Epoch [1/1], Loss: 0.0


In [39]:
model.eval()

# Initialize test metrics
age_preds_list, age_labels_list = [], []
correct_gender, correct_ethnicity, total_samples = 0, 0, 0

# Loss tracking
total_loss = 0.0

with torch.no_grad():
    for images, age_labels, gender_labels, ethnicity_labels in test_dataloader:
        images, age_labels, gender_labels, ethnicity_labels = (
            images.to(device),
            age_labels.float().unsqueeze(1).to(device),
            gender_labels.float().unsqueeze(1).to(device),
            ethnicity_labels.to(device)
        )


        # Model Predictions
        age_preds, gender_preds, ethnicity_preds = model(images)

        # Convert predictions to labels
        # _, predicted_age = torch.max(age_preds, 1)
        predicted_gender = (gender_preds > 0.5).float()
        _, predicted_ethnicity = torch.max(ethnicity_preds, 1)

         # Collect data for MAE calculation
        age_preds_list.extend(age_preds.cpu().numpy().flatten())
        age_labels_list.extend(age_labels.cpu().numpy().flatten())

        # Update correct predictions
        # correct_age += (predicted_age == age_labels).sum().item()
        correct_gender += (predicted_gender == gender_labels).sum().item()
        correct_ethnicity += (predicted_ethnicity == ethnicity_labels).sum().item()

        total_samples += len(test_dataloader)

# Calculate Accuracy
mae_age = mean_absolute_error(age_labels_list, age_preds_list)
gender_accuracy = 100 * correct_gender / total_samples
ethnicity_accuracy = 100 * correct_ethnicity / total_samples

# Print Results
print(f"Age Prediction Error: {mae_age}")
print(f"Gender Prediction Accuracy: {gender_accuracy}%")
print(f"Ethnicity Prediction Accuracy: {ethnicity_accuracy}%")

Age Prediction Accuracy: 15.125213596494458%
Gender Prediction Accuracy: 62.32888888888889%
Ethnicity Prediction Accuracy: 46.84444444444444%
